In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
import re
import pandas as pd
import numpy as np

## 모델링에 사용할 파일
* 불러오기
* null 값 있는지 확인하기
* label 형성하기

In [21]:
df = pd.read_csv("../../data/naver_shopping_preprocessed.csv", encoding = 'utf-8', index_col = 0)

In [22]:
df.isnull().sum()

Score     0
Review    0
dtype: int64

In [23]:
df.head()

,Score,Review
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 박음질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [25]:
def rating_to_label(rating):
    if rating >=4 :
        return 1
    else:
        return 0

df['y'] = df['Score'].apply(lambda x: rating_to_label(x))

## 불용어 사전
* 기본적인 불용어 사전 불러오기
* 자체적으로 추가한 불용어 사전 합치기

In [37]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

In [46]:
temp = []
for s in stopwords:
    temp.append(s[0])

In [47]:
f = open("../../data/stopwords_self.txt", 'r', encoding= 'utf-8')

In [48]:
lines = f.readlines()
for idx, l in enumerate(lines):
    lines[idx] = l.strip()

In [41]:
lines[:5]

['지금', '입다', '입히다', '음', '욤']

In [49]:
temp2 = []
for l in lines:
    temp2.append(l)

In [55]:
stopwords= list(set(temp).union(set(temp2)))

In [58]:
stopwords[:5]

['앞의것', '까지 미치다', '콸콸', '잠', '남']

## 토큰화 함수 정의
* 명사, 형용사, 동사 + 1글자 이상만 추출하는 경우 - tokenizer_1
* 형태소 모두 사용 - tokenizer_2

In [66]:
def tokenizer_1(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]')
    result = hangul.sub('', text)
    okt = Okt()
    Okt_morphs = okt.pos(result)
    words = []

    for word, pos in Okt_morphs:
        if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':
            if len(word) > 1 and word not in stopwords:
                words.append(word)

    return ' '.join(words)

In [67]:
okt2 = Okt()
def tokenizer_2(text):
    tokens_ko = okt2.morphs(text)

    result = []
    for word in tokens_ko:
        if word not in stopwords:
            result.append(word)
    return ' '.join(result)

In [68]:
tokenizer_1("아버지 가방에 들어가신다")

'아버지 가방 들어가신다'

In [69]:
tokenizer_2("아버지 가방에 들어가신다")

'아버지 가방 들어가신다'

In [79]:
def tokenizer(text):
    return text.split()

## Tokenized 된 컬럼을 df에 추가
* 원래는 CounterVectorizer나 TFIDF 하이퍼파라미터에 해당 tokenizer_1, tokenizer_2 함수를 tokenizer로 지정하는 방법이 기본적인 방법
* 시간을 효율적으로 쓰기 위해 tokenizer_1, tokenizer_2 함수를 적용한 컬럼 tokenized_1, tokenized_2를 df에 추가함
* CounterVectorizer와 TFIDF함수에 이미 토큰화된 컬럼을 테스트셋, 훈련용셋으로 나눠서 사용하고 tokenizer는 띄어쓰기 기준으로 split하는 함수 사용

In [70]:
import time
from tqdm import tqdm

tokenized_1 = []

for idx, row in tqdm(df.iterrows()):
    tokenized_review = tokenizer_1(df.loc[idx, 'Review'])
    tokenized_1.append(tokenized_review)

200000it [1:19:32, 41.91it/s] 


In [71]:
df['tokenized_1'] = tokenized_1

In [72]:
tokenized_2 = []

for idx, row in tqdm(df.iterrows()):
    tokenized_review = tokenizer_2(df.loc[idx, 'Review'])
    tokenized_2.append(tokenized_review)

df['tokenized_2'] = tokenized_2

200000it [10:34, 315.20it/s]


In [73]:
df.head()

,Score,Review,y,tokenized_1,tokenized_2
0,5,배공빠르고 굿,1,배공 빠르고,배공 빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0,택배 엉망 놔두고가고,택배 엉망 이네 용 집 밑 층 없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 박음질이 조금 ...,1,아주 좋아요 바지 정말 좋아서 구매 가격 대박 입니다 박음질 어설프다하긴 편하고 가...,아주 좋아요 바지 정말 좋아서 2 구매 가격 대박 입니다 . 박음질 어설프다하긴 편...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0,선물 받아서 전달 했어야 하는 상품 이었는데 머그컵 와서 당황 했습니다 전화했더니 ...,선물 용 빨리 받아서 전달 했어야 하는 상품 이었는데 머그컵 와서 당황 했습니다 ....
4,5,민트색상상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1,민트 색상 예뻐요 손잡이 도로 사용 되네요,민트 색상 예뻐요 . 손잡이 는 는 용 도로 사용 되네요 ㅎㅎ


In [75]:
df.to_csv("../../data/naver_shopping_tokenized_review.csv", sep = ",", encoding = "utf-8")

In [76]:
print(df.isnull().sum())
df.info()

Score          0
Review         0
y              0
tokenized_1    0
tokenized_2    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Score        200000 non-null  int64 
 1   Review       200000 non-null  object
 2   y            200000 non-null  int64 
 3   tokenized_1  200000 non-null  object
 4   tokenized_2  200000 non-null  object
dtypes: int64(2), object(3)
memory usage: 13.2+ MB


## DTM + GridSearchCV + tokenizer_1
* countervectorizer
* GridSearchCV
* Tokenizer1

In [106]:
from sklearn.model_selection import train_test_split
y = df['y']
X_train_texts, X_test_texts, y_train, y_test = train_test_split(df['tokenized_1'], y, test_size = 0.2, random_state = 0)

In [80]:
vect = CountVectorizer(min_df = 3, ngram_range=(1,2), tokenizer = tokenizer)
X_train_tf = vect.fit_transform(X_train_texts)
X_test_tf = vect.transform(X_test_texts)

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
vocablist_DTM_tk1 = [word for word, number in sorted(vect.vocabulary_.items(), key = lambda x:x[1])]

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression(random_state = 0, solver = 'liblinear')
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lr , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(X_train_tf , y_train)
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


{'C': 1} 0.8942


In [82]:
best_estimator_DTM_tk1 = grid_cv.best_estimator_
y_pred = best_estimator_DTM_tk1.predict(X_test_tf)

In [83]:
print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.90
Precision : 0.901
Recall : 0.891
F1 : 0.896


In [84]:
import pickle

with open('DTM_tokenizer1_gridsearch_done.pickle','wb') as fw:
    pickle.dump(best_estimator_DTM_tk1, fw)

In [ ]:
coefficients = best_estimator_DTM_tk1.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[:10]:
    print('{0:}({1:.3f})'.format(vocablist_DTM_tk1[word_num], coef))

print('\n부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[-10:][::-1]: 
    print('{0:}({1:.3f})'.format(vocablist_DTM_tk1[word_num], coef))

## TF-IDF + GridSearchCV + tokenizer_1
* TFIDF
* GridSearchCV
* Tokenizer1

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df = 3, max_df = 0.9, tokenizer = tokenizer)
tfidf_vectorizer.fit(X_train_texts)
tfidf_matrix_train = tfidf_vectorizer.transform(X_train_texts)
tfidf_matrix_test = tfidf_vectorizer.transform(X_test_texts)

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
vocablist_TFIDF_tk1 = [word for word, number in sorted(tfidf_vectorizer.vocabulary_.items(), key = lambda x:x[1])]

In [108]:
lr = LogisticRegression(random_state = 0, solver = 'liblinear')
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lr , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train , y_train)
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8965


In [112]:
best_estimator_TFIDF_tk1 = grid_cv.best_estimator_
y_pred = best_estimator_TFIDF_tk1.predict(tfidf_matrix_test)

In [113]:
print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.90
Precision : 0.901
Recall : 0.893
F1 : 0.897


In [101]:
with open('TFIDF_tokenizer1_gridsearch_done.pickle','wb') as fw:
    pickle.dump(best_estimator_TFIDF_tk1, fw)

In [ ]:
coefficients = best_estimator_TFIDF_tk1.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[:10]:
    print('{0:}({1:.3f})'.format(vocablist_TFIDF_tk1[word_num], coef))

print('\n부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[-10:][::-1]: 
    print('{0:}({1:.3f})'.format(vocablist_TFIDF_tk1[word_num], coef))

## DTM + GridSearchCV + tokenizer_2
* countervectorizer
* GridSearchCV
* Tokenizer2

In [115]:
X_train_texts, X_test_texts, y_train, y_test = train_test_split(df['tokenized_2'], y, test_size = 0.2, random_state = 0)

In [91]:
vect2 = CountVectorizer(min_df = 3, ngram_range=(1,2), tokenizer = tokenizer)
X_train_tf = vect2.fit_transform(X_train_texts)
X_test_tf = vect2.transform(X_test_texts)

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
vocablist_DTM_tk2 = [word for word, number in sorted(vect2.vocabulary_.items(), key = lambda x:x[1])]

In [92]:
lr = LogisticRegression(random_state = 0, solver = 'liblinear')
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lr , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(X_train_tf , y_train)
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn

{'C': 1} 0.9095


In [93]:
best_estimator_DTM_tk2 = grid_cv.best_estimator_
y_pred = best_estimator_DTM_tk2.predict(X_test_tf)

print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.91
Precision : 0.910
Recall : 0.909
F1 : 0.909


In [94]:
with open('DTM_tokenizer2_gridsearch_done.pickle','wb') as fw:
    pickle.dump(best_estimator_TFIDF_tk2, fw)

In [ ]:
coefficients = best_estimator_DTM_tk2.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[:10]:
    print('{0:}({1:.3f})'.format(vocablist_DTM_tk2[word_num], coef))

print('\n부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[-10:][::-1]: 
    print('{0:}({1:.3f})'.format(vocablist_DTM_tk2[word_num], coef))

## TFIDF + GridSearchCV + tokenizer_2
* TFIDF
* GridSearchCV
* Tokenizer2

In [116]:
tfidf_vectorizer2 = TfidfVectorizer(ngram_range=(1,2), min_df = 3, max_df = 0.9, tokenizer = tokenizer)
tfidf_vectorizer2.fit(X_train_texts)
tfidf_matrix_train = tfidf_vectorizer2.transform(X_train_texts)
tfidf_matrix_test = tfidf_vectorizer2.transform(X_test_texts)

In [ ]:
vocablist_TFIDF_tk2 = [word for word, number in sorted(vect2.vocabulary_.items(), key = lambda x:x[1])]

In [117]:
lr = LogisticRegression(random_state = 0, solver = 'liblinear')
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lr , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train, y_train)
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.9123


In [118]:
best_estimator_TFIDF_tk2 = grid_cv.best_estimator_
y_pred = best_estimator_TFIDF_tk2.predict(X_test_tf)
print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.90
Precision : 0.882
Recall : 0.928
F1 : 0.904


In [105]:
with open('TFIDF_tokenizer2_gridsearch_done.pickle','wb') as fw:
    pickle.dump(best_estimator_TFIDF_tk2, fw)

In [ ]:
coefficients = best_estimator_TFIDF_tk2.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# coefficients(계수)가 큰 값부터 내림차순으로 정렬

print('긍정적인 단어 Top 10 (높은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[:10]:
    print('{0:}({1:.3f})'.format(vocablist_TFIDF_tk2[word_num], coef))

print('\n부정적인 단어 Top 10 (낮은 평점과 상관관계가 강한 단어들)')
for word_num, coef in sorted_coefficients[-10:][::-1]: 
    print('{0:}({1:.3f})'.format(vocablist_TFIDF_tk2[word_num], coef))

## DTM + 하이퍼파라미터 튜닝 + 유의어 일원화 + 명사/형용사/동사만 추출
* accuracy: 0.90
* Precision : 0.901
* Recall : 0.891
* F1 : 0.896


## TFIDF + 하이퍼파라미터 튜닝 + 유의어 일원화 + 명사/형용사/동사만 추출
* accuracy: 0.90
* Precision : 0.901
* Recall : 0.893
* F1 : 0.897

## DTM + 하이퍼파라미터 튜닝 + 유의어 일원화 + 형태소만 추출
* accuracy: 0.91
* Precision : 0.910
* Recall : 0.909
* F1 : 0.909


## TFIDF + 하이퍼파라미터 튜닝 + 유의어 일원화 + 형태소만 추출
* accuracy: 0.90
* Precision : 0.882
* Recall : 0.928
* F1 : 0.904